In [1]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import urllib.request
from PIL import Image
from pydataset import data
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from acquire import *
from prepare import *
from explore import *
import env
import warnings
warnings.filterwarnings("ignore")

|               | pred dog   | pred cat   |
|:------------  |-----------:|-----------:|
| actual dog    |         46 |         7  |
| actual cat    |         13 |         34 |


In [3]:
# For this evaluation, I am choosing "dog" to be the positive/affirmitive.

# A false positive would be that the prediction was dog, but it was actually a cat.

# A false negative would be that the prediction was cat, but it was actually a dog.

#True positive is predicting its a dog, and it's a dog.
tp = 46

#True negative is predicting its a cat, and it's a cat.
tn = 34

#False positive is predicting its a dog, but it's a cat.
fp = 13

#false negative is predicting its a cat, but it's a dog.
fn = 7

print("Dog-classifier (where 'dog' is the positive prediction)")

print("True Positives", tp)
print("False Positives", fp)
print("False Negatives", fn)
print("True Negatives", tn)

print("-------------")

accuracy = (tp + tn) / (tp + tn + fp + fn)

precision = tp / (tp + fp)

recall = tp / (tp + fn)

print("Accuracy is", accuracy)
print("Recall is", round(recall,2))
print("Precision is", round(precision,2))


Dog-classifier (where 'dog' is the positive prediction)
True Positives 46
False Positives 13
False Negatives 7
True Negatives 34
-------------
Accuracy is 0.8
Recall is 0.87
Precision is 0.78


In [3]:
df = prep_tit(pd.DataFrame(new_tit_df()))



,passenger_id,survived,pclass,sex,sibsp,parch,fare,embark_town,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
0,0,0,3,male,1,0,7.2500,Southampton,0,1,0,1
1,1,1,1,female,1,0,71.2833,Cherbourg,0,0,0,0
2,2,1,3,female,0,0,7.9250,Southampton,1,0,0,1
